In [3]:
from parse import parse
import sys
sys.path.append('../Train')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import tensorflow as tf
import mnist_inference
import re
import os

### pretreatment predict_set

In [4]:
trainset_df_path = '../CleanData/trainDataAfterClean.csv'
predict_df_path = '../RawData/yancheng_testA_20171225.csv'

In [5]:
trainset_df = pd.read_csv(trainset_df_path, delimiter=',')
predict_df = pd.read_csv(predict_df_path, delimiter=',')

In [6]:
facet_filter_pattern = re.compile('(sale_quantity)|(year)|(month)')
year_month_pattern = re.compile('(year)|(month)')

facet_columns = [c for c in trainset_df.columns.values if not facet_filter_pattern.match(c)]
class_id_columns = [c for c in trainset_df.columns.values if c.find('class_id') > -1]
year_month_columns = [c for c in trainset_df.columns.values if year_month_pattern.match(c)]

In [7]:
unique_trainset_df = trainset_df.drop_duplicates(facet_columns)

In [8]:
predict_df['year'] = predict_df['predict_date'].astype('str').apply(lambda x:int(x[:4]))
predict_df['month'] = predict_df['predict_date'].astype('str').apply(lambda x:int(x[4:]))
predict_df = pd.get_dummies(predict_df.drop(['predict_date'], axis=1), columns=['class_id', 'year', 'month'])
predict_df = pd.merge(predict_df, unique_trainset_df[facet_columns], on=class_id_columns)
for date_c in set(year_month_columns)-set([c for c in predict_df.columns.values if year_month_pattern.match(c)]):
    predict_df.loc[:, date_c] = pd.Series(np.zeros(len(predict_df)), index=predict_df.index)
predict_df = predict_df.reindex(columns=trainset_df.columns.values)

In [9]:
predict_df.head()

,sale_quantity,compartment,level_id,TR,displacement,if_charging,price_level,if_MPV_id,if_luxurious_id,power,...,newenergy_type_id_4,emission_standards_id_1,emission_standards_id_2,emission_standards_id_3,emission_standards_id_5,gearbox_type_AMT,gearbox_type_AT,gearbox_type_CVT,gearbox_type_DCT,gearbox_type_MT
0,NaN,2,3.0,6.0,1.6,1,12.5,1,0,91.6,...,0,1,0,0,0,0,0,0,0,1
1,NaN,2,3.0,6.0,1.6,1,12.5,1,0,91.6,...,0,1,0,0,0,0,0,1,0,0
2,NaN,2,3.0,6.0,1.6,1,12.5,1,0,91.6,...,0,1,0,0,0,0,0,0,0,1
3,NaN,2,3.0,7.0,1.6,0,12.5,1,0,117.7,...,0,1,0,0,0,0,1,0,0,0
4,NaN,2,3.0,6.0,1.6,1,12.5,1,0,91.6,...,0,1,0,0,0,0,1,0,0,0


### predict

In [13]:
MODEL_SAVE_PATH = "../model/2018-01-22/"
MODEL_NAME = "tianchi_model"

In [15]:
def predict_on_model_1(df):
    
    def normalize(df):
        normalizeColumns = ['compartment','TR','displacement','price_level','power','level_id',
                    'cylinder_number','engine_torque','car_length','car_height','car_width','total_quality','equipment_quality',
                    'rated_passenger','wheelbase','front_track','rear_track']
        leftDf = df.drop(normalizeColumns, axis =1 )
        normalizeDf = df[normalizeColumns]
        normalizeDf = (normalizeDf-normalizeDf.min())/(normalizeDf.max()-normalizeDf.min())
        inputDf_1 = pd.concat([leftDf, normalizeDf], axis = 1)
        return inputDf_1.values
    
    config = tf.ConfigProto(allow_soft_placement = True, log_device_placement=True)
    restoredSession = tf.Session(config=config)
    restoredSaver = tf.train.import_meta_graph(os.path.join(MODEL_SAVE_PATH, 'tianchi_model-34500.meta'))
    restoredSaver.restore(restoredSession, os.path.join(MODEL_SAVE_PATH, 'tianchi_model-34500'))
    w_1 = restoredSession.run('layer1/weights:0')
    B_1 = restoredSession.run('layer1/biases:0')
    w_2 = restoredSession.run('layer2/weights:0')
    b_2 = restoredSession.run('layer2/biases:0')

    x = tf.placeholder(tf.float32, [None, mnist_inference.INPUT_NODE], name='x-input')
    input_tensor = restoredSession.run(x, feed_dict={x: normalize(df)})
    layer1 = tf.nn.relu(tf.matmul(input_tensor, w_1) + B_1)
    layer2 = tf.nn.relu(tf.matmul(layer1, w_2) + b_2)

    return restoredSession.run(layer2)

In [16]:
predict_quantity = predict_on_model_1(predict_df.drop(['sale_quantity'], axis=1))
predict_df['predict_quantity'] = predict_quantity
predict_df['class_id'] = predict_df.apply(lambda row: (row[class_id_columns]==1).idxmax(axis=1)[9:], axis=1).astype('int64')
predict_result = predict_df.groupby(['class_id'])['predict_quantity'].mean()

INFO:tensorflow:Restoring parameters from ../model/2018-01-22/tianchi_model-34500


In [17]:
predicted_df = pd.read_csv(predict_df_path, delimiter=',')
predicted_df = predicted_df.set_index('class_id')
predicted_df.update(predict_result)
predicted_df

,predict_date,predict_quantity
class_id,,
103507,201711,451.347137
124140,201711,300.531860
125403,201711,374.505402
136916,201711,325.801178
169673,201711,160.751648
175962,201711,216.238235
178529,201711,671.778015
186250,201711,155.988571
194201,201711,550.141968


In [20]:
predicted_df.to_csv('../Predict/predict_result/20180122/yancheng_testA_20171225.csv')

In [21]:
import numpy as np

In [26]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b

In [25]:
c.shape

(3, 3)